# Coal Inventory Scraper for Chinese Ports

This notebook contains a Python script that scrapes coal inventory data from the sxcoal.com website. It specifically targets articles containing "煤炭库存" (coal inventory) to extract date, port name, and inventory figures. The extracted data is then reshaped into a table and saved as a CSV file.

In [2]:
## need to install undetected_chromedriver and pandas if not already installed
# This script scrapes coal inventory data from a specific website, reshapes it, and exports
# the final table to a CSV file. It uses Selenium for web scraping and Pandas for data manipulation.
# File: coal_inventory_scraper.py
import pandas as pd
import re
from datetime import datetime
import time

try:
    import undetected_chromedriver as uc
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import TimeoutException
except ImportError:
    print("Required libraries not found. Installing them now...")
    %pip install undetected-chromedriver pandas
    import undetected_chromedriver as uc
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import TimeoutException

def scrape_export_inventory_data():
    """
    Final version: Scrapes all inventory articles, reshapes the data,
    and exports the final table to a CSV file.
    """
    search_url = "https://www.sxcoal.com/news/search?search=%E7%85%A4%E7%82%AD%E5%BA%93%E5%AD%98"
    csv_filename = "coal_inventory_data.csv"
    raw_data_list = []

    print("--- Initializing browser for scrape-and-export task ---")
    
    driver = None
    try:
        options = uc.ChromeOptions()
        options.add_argument('--headless')
        
        driver = uc.Chrome(options=options, use_subprocess=False)
        wait = WebDriverWait(driver, 25) 

        # Step 1: Gather URLs
        print(f"Navigating to: {search_url}")
        driver.get(search_url)

        print("\nFinding all relevant inventory articles...")
        article_link_selector = (By.PARTIAL_LINK_TEXT, "煤炭库存")
        
        try:
            wait.until(EC.element_to_be_clickable(article_link_selector))
            article_elements = driver.find_elements(*article_link_selector)
            article_urls = sorted(list(set([el.get_attribute('href') for el in article_elements])))
            print(f">>> Success! Found {len(article_urls)} unique articles to process.")
        except TimeoutException:
            print("\n--- FAILURE ---")
            print("Could not find any article links containing '煤炭库存'.")
            return None 

        # Step 2: Loop through URLs and extract data
        for i, url in enumerate(article_urls):
            print(f"\n--- Processing article {i+1} of {len(article_urls)} ---")
            try:
                driver.get(url)
                time.sleep(3) 
                page_text = driver.find_element(By.TAG_NAME, 'body').text
                pattern = re.compile(r"(\d+月\d+日)，([\u4e00-\u9fa5]+港)煤炭库存为(\d+\.?\d*)")
                match = pattern.search(page_text)
                
                if match:
                    date_str, port_name, inventory = match.groups()
                    raw_data_list.append({"Date": date_str, "Port": port_name.strip(), "Inventory": float(inventory)})
                    print(f"  >>> SUCCESS: Found [{date_str}] '{port_name.strip()}' with inventory {inventory}")
                else:
                    print(f"  > INFO: No data matching the required pattern was found.")
                    
            except Exception as e:
                print(f"  > ERROR: An error occurred processing this article: {type(e).__name__}")
        
        if not raw_data_list:
            print("\nScraping finished, but no data could be extracted.")
            return None
            
        # Step 3: Reshape the data
        print("\n--- Data extraction complete. Reshaping table... ---")
        long_df = pd.DataFrame(raw_data_list)
        current_year = datetime.now().year
        long_df['FullDate'] = long_df['Date'].apply(lambda x: datetime.strptime(f'{current_year}年{x}', '%Y年%m月%d日'))

        final_table = long_df.pivot_table(index='FullDate', columns='Port', values='Inventory')
        final_table.sort_index(ascending=False, inplace=True)
        final_table.reset_index(inplace=True)
        final_table.rename(columns={'FullDate': 'Date'}, inplace=True)
        final_table['Date'] = final_table['Date'].dt.strftime('%m-%d-%y')
        
        # ***4: EXPORT TO CSV ***
        print(f"\n--- Reshaping complete. Exporting data to '{csv_filename}'... ---")
        try:
            # We use index=False to avoid writing the pandas row numbers (0, 1, 2...) into the file.
            # encoding='utf-8-sig' helps Excel open the file correctly with Chinese characters.
            final_table.to_csv(csv_filename, index=False, encoding='utf-8-sig')
            print(f">>> SUCCESS: Data has been saved to {csv_filename}")
        except Exception as e:
            print(f"--- FAILED TO SAVE FILE: An error occurred: {e} ---")

        return final_table

    except Exception as e:
        print(f"\nA critical error occurred: {type(e).__name__} - {e}")
        return None
    finally:
        if driver:
            print("\nScript finished. Closing browser...")
            driver.quit()

# --- Run the scraper ---
final_inventory_data = scrape_export_inventory_data()

# --- Display the final DataFrame in the notebook for confirmation ---
if final_inventory_data is not None and not final_inventory_data.empty:
    print("\n\n--- FINAL DATA PREVIEW ---")
    display(final_inventory_data) 
else:
    print("\n\n--- TASK FAILED OR NO DATA FOUND ---")

--- Initializing browser for scrape-and-export task ---
Navigating to: https://www.sxcoal.com/news/search?search=%E7%85%A4%E7%82%AD%E5%BA%93%E5%AD%98
Navigating to: https://www.sxcoal.com/news/search?search=%E7%85%A4%E7%82%AD%E5%BA%93%E5%AD%98

Finding all relevant inventory articles...
>>> Success! Found 16 unique articles to process.

--- Processing article 1 of 16 ---

Finding all relevant inventory articles...
>>> Success! Found 16 unique articles to process.

--- Processing article 1 of 16 ---
  >>> SUCCESS: Found [7月2日] '秦皇岛港' with inventory 580

--- Processing article 2 of 16 ---
  >>> SUCCESS: Found [7月2日] '秦皇岛港' with inventory 580

--- Processing article 2 of 16 ---
  >>> SUCCESS: Found [7月2日] '黄骅港' with inventory 175.6

--- Processing article 3 of 16 ---
  >>> SUCCESS: Found [7月2日] '黄骅港' with inventory 175.6

--- Processing article 3 of 16 ---
  >>> SUCCESS: Found [7月3日] '秦皇岛港' with inventory 580

--- Processing article 4 of 16 ---
  >>> SUCCESS: Found [7月3日] '秦皇岛港' with inve

Port,Date,京唐港,曹妃甸港,秦皇岛港,黄骅港
0,07-09-25,701.0,1254.0,573.0,189.3
1,07-08-25,NaN,NaN,575.0,182.5
2,07-07-25,NaN,NaN,576.0,NaN
3,07-04-25,NaN,NaN,570.0,NaN
4,07-03-25,NaN,NaN,580.0,180.3
5,07-02-25,NaN,NaN,580.0,175.6
